In [5]:
data=[]

In [6]:
data.append(
    {
        "image": "C:/AI/Github/Reconnaissance_drone_report/Data/Images/earthquake/download.jpg",
        "text": "Describe the image.",
        "content": "This image shows a distructed building due to earthquake since there is a lot of debris is setteled in this area"
    }
)

In [9]:
from transformers import AutoProcessor, AutoModelForVision2Seq
from peft import get_peft_model, LoraConfig, TaskType
import torch

model_id = "Qwen/Qwen-VL"  # or Qwen/Qwen-VL-Chat

processor = AutoProcessor.from_pretrained(model_id,trust_remote_code=True)
model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # May vary depending on Qwen2's layer names
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
from torch.utils.data import DataLoader
from tqdm import tqdm

model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
dataloader = DataLoader(data, batch_size=1, shuffle=True)

for epoch in range(3):
    for batch in tqdm(dataloader):
        batch = {k: v.to("cuda") for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


ValueError: Unrecognized configuration class <class 'transformers_modules.Qwen.Qwen-VL.0547ed36a86561e2e42fecec8fd0c4f6953e33c4.configuration_qwen.QWenConfig'> for this kind of AutoModel: AutoModelForVision2Seq.
Model type should be one of BlipConfig, Blip2Config, ChameleonConfig, GitConfig, Idefics2Config, Idefics3Config, InstructBlipConfig, InstructBlipVideoConfig, Kosmos2Config, LlavaConfig, LlavaNextConfig, LlavaNextVideoConfig, LlavaOnevisionConfig, Mistral3Config, MllamaConfig, PaliGemmaConfig, Pix2StructConfig, Qwen2_5_VLConfig, Qwen2VLConfig, VideoLlavaConfig, VipLlavaConfig, VisionEncoderDecoderConfig.

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!pip install matplotlib tiktoken